In [35]:
import glob

import pandas as pd
import numpy as np

from joblib import Parallel, delayed, Memory

from skcycling.io import bikeread
from skcycling.extraction import acceleration
from skcycling.extraction import gradient_activity
from skcycling.extraction import gradient_elevation
from skcycling.extraction import gradient_heart_rate

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import QuantileTransformer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import cross_validate

In [2]:
path_data = '/home/lemaitre/Documents/data/cycling/user_2/2014/*.fit'
filenames = sorted(glob.glob(path_data))
filenames[:5]

['/home/lemaitre/Documents/data/cycling/user_2/2014/2014-01-04-11-40-46.fit',
 '/home/lemaitre/Documents/data/cycling/user_2/2014/2014-01-05-08-15-15.fit',
 '/home/lemaitre/Documents/data/cycling/user_2/2014/2014-01-07-15-09-05.fit',
 '/home/lemaitre/Documents/data/cycling/user_2/2014/2014-01-08-15-05-10.fit',
 '/home/lemaitre/Documents/data/cycling/user_2/2014/2014-01-09-15-09-11.fit']

Cache the results of the reading to avoid losing time to reload data

In [3]:
memory = Memory(location='bikereadcache')
bikeread_cached = memory.cache(bikeread, verbose=1)

In [5]:
data = Parallel(n_jobs=-1)(delayed(bikeread_cached)(f) for f in filenames)

Only keep the data with all information

In [8]:
# filter the activity which do not contain the required information
fields = ['elevation', 'cadence', 'distance', 'heart-rate', 'power', 'speed']

valid_data = []
for activity in data:
    if set(fields).issubset(activity.columns):
        if not pd.isnull(activity).any().any():
            valid_data.append(activity)

In [9]:
len(valid_data)

52

In [10]:
len(data)

122

In [11]:
data = valid_data

In [12]:
data[0].head()

,elevation,cadence,distance,heart-rate,power,speed
2014-01-04 10:40:46,110.4,33.0,2.85,112.0,70.0,2.854
2014-01-04 10:40:47,110.4,35.0,5.96,112.0,51.0,3.110
2014-01-04 10:40:48,110.4,38.0,9.26,113.0,60.0,3.297
2014-01-04 10:40:49,110.4,40.0,12.77,113.0,50.0,3.506
2014-01-04 10:40:50,110.0,50.0,16.46,113.0,63.0,3.669


Compute the elevation, speed, and heart-rate gradient. In addition, we will add information about the 10 past record using the derivative.

In [13]:
for activity_idx in range(len(data)):
    # compute acceleration
    data[activity_idx] = acceleration(data[activity_idx])
    # compute gradient elevation
    data[activity_idx] = gradient_elevation(data[activity_idx])
    # compute gradient heart-rate
    data[activity_idx] = gradient_heart_rate(data[activity_idx])
    # compute the gradient information over 10 sec for the some fields
    fields = ['elevation', 'cadence', 'heart-rate', 'speed',
              'gradient-elevation', 'gradient-heart-rate', 'acceleration']
    data[activity_idx] = gradient_activity(data[activity_idx], periods=range(1, 11), columns=fields)

In [14]:
data[0]

original                                             \
                    elevation cadence  distance heart-rate  power  speed   
2014-01-04 10:40:46     110.4    33.0      2.85      112.0   70.0  2.854   
2014-01-04 10:40:47     110.4    35.0      5.96      112.0   51.0  3.110   
2014-01-04 10:40:48     110.4    38.0      9.26      113.0   60.0  3.297   
2014-01-04 10:40:49     110.4    40.0     12.77      113.0   50.0  3.506   
2014-01-04 10:40:50     110.0    50.0     16.46      113.0   63.0  3.669   
2014-01-04 10:40:51     110.0    63.0     20.27      114.0    1.0  3.834   
2014-01-04 10:40:52     110.0    50.0     24.03      113.0   25.0  3.763   
2014-01-04 10:40:53     110.0    50.0     28.13      114.0  251.0  4.099   
2014-01-04 10:40:54     110.0    85.0     32.15      115.0   58.0  4.015   
2014-01-04 10:40:55     110.0   124.0     36.43      115.0    1.0  4.285   
2014-01-04 10:40:56     110.0    34.0     41.10      116.0    0.0  4.670   
2014-01-04 10:40:57     110.0     0.0     45.77      116.0    0.0  4.670   
2014-01-04 10:40:58     110.0     0.0     50.27      115.0   16.0  4.502   
2014-01-04 10:40:59     110.0    74.0     54.74      115.0  273.0  4.464   
2014-01-04 10:41:00     110.0    78.0     59.04      115.0  291.0  4.302   
2014-01-04 10:41:01     110.0    72.0     63.44      116.0  264.0  4.400   
2014-01-04 10:41:02     110.0    75.0     68.55      117.0  219.0  5.112   
2014-01-04 10:41:03     110.0   100.0     74.40      119.0    2.0  5.849   
2014-01-04 10:41:04     110.0    27.0     80.55      120.0   57.0  6.135   
2014-01-04 10:41:05     110.0    89.0     86.67      120.0  120.0  6.135   
2014-01-04 10:41:06     110.0    92.0     92.48      120.0  131.0  5.809   
2014-01-04 10:41:07     110.0    80.0     98.36      120.0  306.0  5.881   
2014-01-04 10:41:08     110.0    74.0    104.18      121.0  228.0  5.817   
2014-01-04 10:41:09     110.0    75.0    110.04      121.0   87.0  5.865   
2014-01-04 10:41:10     110.0    91.0    116.64      121.0    0.0  6.600   
2014-01-04 10:41:11     110.0    51.0    123.12      121.0    0.0  6.470   
2014-01-04 10:41:12     110.0     0.0    129.33      122.0    0.0  6.197   
2014-01-04 10:41:13     110.0     0.0    135.51      123.0    0.0  6.197   
2014-01-04 10:41:14     110.0     0.0    141.52      124.0    0.0  6.014   
2014-01-04 10:41:15     110.0     0.0    147.01      125.0    0.0  5.494   
...                       ...     ...       ...        ...    ...    ...   
2014-01-04 12:56:46      94.0    63.0  68653.00      156.0  269.0  2.556   
2014-01-04 12:56:47      94.0    48.0  68656.42      156.0  174.0  3.450   
2014-01-04 12:56:48      94.4    42.0  68660.33      156.0  158.0  3.907   
2014-01-04 12:56:49      95.0    43.0  68664.45      155.0  150.0  4.113   
2014-01-04 12:56:50      95.4    46.0  68668.49      155.0  159.0  4.043   
2014-01-04 12:56:51      95.8    47.0  68672.88      155.0  180.0  4.387   
2014-01-04 12:56:52      96.4    51.0  68677.38      154.0    1.0  4.498   
2014-01-04 12:56:53      96.4    55.0  68682.06      154.0    1.0  4.654   
2014-01-04 12:56:54      96.4     0.0  68686.68      154.0    0.0  4.654   
2014-01-04 12:56:55      96.4     0.0  68690.96      154.0    0.0  4.273   
2014-01-04 12:56:56      96.4     0.0  68695.17      154.0    5.0  4.223   
2014-01-04 12:56:57      96.4     0.0  68699.08      153.0    2.0  3.900   
2014-01-04 12:56:58      96.4     0.0  68702.77      153.0    0.0  3.691   
2014-01-04 12:56:59      96.4     0.0  68706.33      152.0    0.0  3.559   
2014-01-04 12:57:00      96.4     0.0  68709.60      152.0    0.0  3.278   
2014-01-04 12:57:01      96.4     0.0  68712.84      151.0   51.0  3.236   
2014-01-04 12:57:02      96.4    38.0  68716.08      151.0   91.0  3.236   
2014-01-04 12:57:03      96.4    63.0  68719.46      150.0   23.0  3.387   
2014-01-04 12:57:04      96.4    51.0  68723.03      150.0    1.0  3.571   
2014-01-04 12:57:05      96.4    42.0  68726.60      150.0    0.0  3.

In [43]:
for activity in data:
    activity.replace([np.inf, -np.inf], np.nan, inplace=True)
    activity.fillna(activity.mean(), inplace=True)

In [44]:
data[0].head()

original                                           \
                    elevation cadence distance heart-rate power  speed   
2014-01-04 10:40:46     110.4    33.0     2.85      112.0  70.0  2.854   
2014-01-04 10:40:47     110.4    35.0     5.96      112.0  51.0  3.110   
2014-01-04 10:40:48     110.4    38.0     9.26      113.0  60.0  3.297   
2014-01-04 10:40:49     110.4    40.0    12.77      113.0  50.0  3.506   
2014-01-04 10:40:50     110.0    50.0    16.46      113.0  63.0  3.669   

                                                                         \
                    acceleration gradient-elevation gradient-heart-rate   
2014-01-04 10:40:46    -0.001179           0.004354            0.019304   
2014-01-04 10:40:47    -0.001179           0.004354            0.019304   
2014-01-04 10:40:48    -0.001179           0.004354            0.019304   
2014-01-04 10:40:49    -0.001179           0.004354            0.019304   
2014-01-04 10:40:50    -0.001179           0.004354            0.019304   

                    gradient_1     ...              gradient_9  \
                     elevation     ...      gradient-elevation   
2014-01-04 10:40:46   -0.00171     ...                -0.00003   
2014-01-04 10:40:47    0.00000     ...                -0.00003   
2014-01-04 10:40:48    0.00000     ...                -0.00003   
2014-01-04 10:40:49    0.00000     ...                -0.00003   
2014-01-04 10:40:50   -0.40000     ...                -0.00003   

                                                     gradient_10            \
                    gradient-heart-rate acceleration   elevation   cadence   
2014-01-04 10:40:46           -0.005137    -0.002028   -0.016822 -0.069438   
2014-01-04 10:40:47           -0.005137    -0.002028   -0.016822 -0.069438   
2014-01-04 10:40:48           -0.005137    -0.002028   -0.016822 -0.069438   
2014-01-04 10:40:49           -0.005137    -0.002028   -0.016822 -0.069438   
2014-01-04 10:40:50           -0.005137    -0.002028   -0.016822 -0.069438   

                                                             \
                    heart-rate     speed gradient-elevation   
2014-01-04 10:40:46   0.038875 -0.001875          -0.000028   
2014-01-04 10:40:47   0.038875 -0.001875          -0.000028   
2014-01-04 10:40:48   0.038875 -0.001875          -0.000028   
2014-01-04 10:40:49   0.038875 -0.001875          -0.000028   
2014-01-04 10:40:50   0.038875 -0.001875          -0.000028   

                                                      
                    gradient-heart-rate acceleration  
2014-01-04 10:40:46           -0.005505    -0.002005  
2014-01-04 10:40:47           -0.005505    -0.002005  
2014-01-04 10:40:48           -0.005505    -0.002005  
2014-01-04 10:40:49           -0.005505    -0.002005  
2014-01-04 10:40:50           -0.005505    -0.002005  

[5 rows x 79 columns]

Concatenate all the information and keep track of the group to avoid leaking

In [45]:
data_concat = pd.concat(data)
y = data_concat['original']['power']
X = data_concat.drop('power', axis=1, level=1)
groups = []
for group_idx, activity in enumerate(data):
    groups += [group_idx] * activity.shape[0]
groups = np.array(groups)

In [46]:
group

array([ 0,  0,  0, ..., 51, 51, 51])

In [47]:
X

original                                      \
                    elevation cadence  distance heart-rate  speed   
2014-01-04 10:40:46     110.4    33.0      2.85      112.0  2.854   
2014-01-04 10:40:47     110.4    35.0      5.96      112.0  3.110   
2014-01-04 10:40:48     110.4    38.0      9.26      113.0  3.297   
2014-01-04 10:40:49     110.4    40.0     12.77      113.0  3.506   
2014-01-04 10:40:50     110.0    50.0     16.46      113.0  3.669   
2014-01-04 10:40:51     110.0    63.0     20.27      114.0  3.834   
2014-01-04 10:40:52     110.0    50.0     24.03      113.0  3.763   
2014-01-04 10:40:53     110.0    50.0     28.13      114.0  4.099   
2014-01-04 10:40:54     110.0    85.0     32.15      115.0  4.015   
2014-01-04 10:40:55     110.0   124.0     36.43      115.0  4.285   
2014-01-04 10:40:56     110.0    34.0     41.10      116.0  4.670   
2014-01-04 10:40:57     110.0     0.0     45.77      116.0  4.670   
2014-01-04 10:40:58     110.0     0.0     50.27      115.0  4.502   
2014-01-04 10:40:59     110.0    74.0     54.74      115.0  4.464   
2014-01-04 10:41:00     110.0    78.0     59.04      115.0  4.302   
2014-01-04 10:41:01     110.0    72.0     63.44      116.0  4.400   
2014-01-04 10:41:02     110.0    75.0     68.55      117.0  5.112   
2014-01-04 10:41:03     110.0   100.0     74.40      119.0  5.849   
2014-01-04 10:41:04     110.0    27.0     80.55      120.0  6.135   
2014-01-04 10:41:05     110.0    89.0     86.67      120.0  6.135   
2014-01-04 10:41:06     110.0    92.0     92.48      120.0  5.809   
2014-01-04 10:41:07     110.0    80.0     98.36      120.0  5.881   
2014-01-04 10:41:08     110.0    74.0    104.18      121.0  5.817   
2014-01-04 10:41:09     110.0    75.0    110.04      121.0  5.865   
2014-01-04 10:41:10     110.0    91.0    116.64      121.0  6.600   
2014-01-04 10:41:11     110.0    51.0    123.12      121.0  6.470   
2014-01-04 10:41:12     110.0     0.0    129.33      122.0  6.197   
2014-01-04 10:41:13     110.0     0.0    135.51      123.0  6.197   
2014-01-04 10:41:14     110.0     0.0    141.52      124.0  6.014   
2014-01-04 10:41:15     110.0     0.0    147.01      125.0  5.494   
...                       ...     ...       ...        ...    ...   
2014-05-04 13:05:12      64.0    71.0  57304.01      158.0  4.476   
2014-05-04 13:05:13      64.0    62.0  57308.80      167.0  4.788   
2014-05-04 13:05:14      64.0    65.0  57313.90      176.0  5.097   
2014-05-04 13:05:15      64.0    67.0  57319.17      176.0  5.267   
2014-05-04 13:05:16      64.0    69.0  57324.53      176.0  5.359   
2014-05-04 13:05:17      64.0    70.0  57330.00      176.0  5.476   
2014-05-04 13:05:18      64.0    72.0  57335.61      176.0  5.606   
2014-05-04 13:05:19      64.0    75.0  57341.21      180.0  5.606   
2014-05-04 13:05:20      64.0     0.0  57346.68      180.0  5.462   
2014-05-04 13:05:21      64.0     0.0  57351.90      180.0  5.222   
2014-05-04 13:05:22      64.0     0.0  57356.90      180.0  5.007   
2014-05-04 13:05:23      64.0     0.0  57361.85      184.0  4.943   
2014-05-04 13:05:24      64.0     0.0  57366.53      189.0  4.678   
2014-05-04 13:05:25      64.0     0.0  57370.53      189.0  4.009   
2014-05-04 13:05:26      64.0    72.0  57374.12      189.0  3.585   
2014-05-04 13:05:27      64.0    54.0  57378.13      189.0  4.009   
2014-05-04 13:05:28      64.0    58.0  57382.58      189.0  4.453   
2014-05-04 13:05:29      64.0    87.0  57387.58      191.0  4.995   
2014-05-04 13:05:30      64.0     0.0  57392.44      191.0  4.859   
2014-05-04 13:05:31      64.0     0.0  57397.32      191.0  4.887   
2014-05-04 13:05:32      64.0     0.0  57402.15      191.0  4.831   
2014-05-04 13:05:33      64.0     0.0  57407.01      191.0  4.853   
2014-05-04 13:05:34      64.0     0.0  57411.72      188.0  4.714   
2014-05-04 13:05:35      64.0     0.0  57416.19      188.0  4.472   
2014-05-04 13:05:36      64.0     0.0  57420.24      189.0  4.047   
2014-05-04 13:05:37

In [48]:
pipe = make_pipeline(QuantileTransformer(), GradientBoostingRegressor(random_state=42))
scores = cross_validate(pipe, X, y, groups=groups, scoring=['r2', 'neg_median_absolute_error'],
                        cv=GroupKFold(n_splits=3), n_jobs=1, return_train_score=True)

In [49]:
scores

{'fit_time': array([ 192.9417398 ,  193.04851174,  184.49550223]),
 'score_time': array([ 3.6782012 ,  4.08030486,  4.14024472]),
 'test_neg_median_absolute_error': array([-31.02544704, -28.34900344, -30.29612706]),
 'test_r2': array([ 0.64734564,  0.6027255 ,  0.59363491]),
 'train_neg_median_absolute_error': array([-29.22017815, -26.87621544, -27.39974912]),
 'train_r2': array([ 0.62038711,  0.65533043,  0.65137509])}